In [1]:
import os
import sys
import time
import random
import pickle as pickle
import numpy as np
import tensorflow as tf
from utils import *
from nn import *

from tqdm import tqdm

/home/aind2/anaconda3/envs/aind2/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [234]:
# import all the data
data = load_10_people()
X_train_ohe = data['X_train'].astype(np.float64)
X_train_nums = X_train_ohe.argmax(-1)
Y_train = data['Y_train'].astype(np.float64)

Loading 1004399 tweets from 4391 unique users.
Loading Twitter dataset took 3 seconds.
Number of Tweets: 97728
Only keeping characters that appear at least 100 times in the corpus
Character set consists of 246 characters
Building X...
Building Y...
Splitting Data...
79159 train char sequences
9773 test char sequences
8796 validation char sequences


In [268]:
# Create the encoder RNN with one GRU cell
vocab_size = 246
embedding_size = 100
max_seq_length = 140
n_layers = 1
batch_size = 50
dropout_rate = 0.5
dim_y = 10
dim_h = 100
learning_rate = 0.001

In [179]:
def create_cell(dim, n_layers, dropout):
    cell = tf.nn.rnn_cell.GRUCell(dim)
    cell = tf.nn.rnn_cell.DropoutWrapper(cell,
        input_keep_prob=dropout)
    if n_layers > 1:
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * n_layers)
    return cell

def linear(inp, dim_out, scope, reuse=False):
    dim_in = inp.get_shape().as_list()[-1]
    with tf.variable_scope(scope) as vs:
        if reuse:
            vs.reuse_variables()

        W = tf.get_variable('W', [dim_in, dim_out])
        b = tf.get_variable('b', [dim_out])
    return tf.matmul(inp, W) + b

In [273]:
tf.reset_default_graph()

# Inputs
encoder_inputs = tf.placeholder(shape = [batch_size, max_seq_length], name = "input_sentences", dtype=tf.int32)
targets = tf.placeholder(shape = [batch_size, max_seq_length, vocab_size], name = "target_sentences", dtype=tf.int32)
labels = tf.placeholder(shape = [batch_size], name = 'labels', dtype=tf.int32)

labels = tf.reshape(labels, [-1, 1])

In [355]:
encoder_inputs
targets
labels

<tf.Tensor 'Reshape:0' shape=(50, 1) dtype=int32>

In [274]:
# Embedding matrices
# TensorShape([Dimension(246), Dimension(100)])
embedding_encoder = tf.get_variable("embedding_matrix", [vocab_size, embedding_size])
embedding_decoder = tf.get_variable("output_embedding_matrix", [vocab_size, embedding_size])
embedding_encoder

<tf.Variable 'embedding_matrix:0' shape=(246, 100) dtype=float32_ref>

In [275]:
# Extract Embeddings
#TensorShape([Dimension(140), Dimension(50), Dimension(100)])
encoder_emb_inp = tf.nn.embedding_lookup(embedding_encoder, encoder_inputs, name = "encoder_embeddings")
encoder_emb_out = tf.nn.embedding_lookup(embedding_decoder, encoder_inputs, name = "decoder_embeddings")
encoder_emb_inp

<tf.Tensor 'encoder_embeddings:0' shape=(50, 140, 100) dtype=float32>

In [276]:
# Encoder Net
cell_e = create_cell(dim_h, n_layers, dropout_rate)
_, z = tf.nn.dynamic_rnn(cell_e, encoder_emb_inp, dtype = tf.float32, scope='encoder')

In [277]:
# Decoder Net
cell_g = create_cell(dim_h, n_layers, dropout_rate)
g_outputs, _ = tf.nn.dynamic_rnn(cell_g, encoder_emb_out,
            initial_state = z, scope='generator')

In [278]:
# take the output and predict words
with tf.variable_scope('projection'):
            proj_W = tf.get_variable('W', [dim_h, vocab_size])
            proj_b = tf.get_variable('b', [vocab_size])
            
g_outputs = tf.nn.dropout(g_outputs, dropout_rate)
g_outputs = tf.reshape(g_outputs, [-1, dim_h])
g_logits = tf.matmul(g_outputs, proj_W) + proj_b
g_logits

<tf.Tensor 'add:0' shape=(7000, 246) dtype=float32>

In [318]:
# reconstruction error
loss_g = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.reshape(targets, [-1, vocab_size]), logits=g_logits)
loss_g = tf.reduce_sum(loss_g) / tf.to_float(batch_size)

preds = tf.argmax(tf.nn.softmax(g_logits), -1)
goalz = tf.cast(tf.reshape(encoder_inputs, [-1]), tf.int64)
accuracy = tf.reduce_mean(tf.cast(tf.equal(preds, goalz), tf.float32))
loss_g = tf.reduce_sum(loss_g) / tf.to_float(batch_size)
loss_g

<tf.Tensor 'truediv_17:0' shape=() dtype=float32>

In [320]:
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_g)

In [ ]:
sess = tf.Session()
sess.run([tf.global_variables_initializer()])
for i in tqdm(range(0, len(X_train-50))):
    _, l, a = sess.run([optimizer, loss_g, accuracy], feed_dict={encoder_inputs: X_train_nums[i:i+50], 
                                                   targets: X_train_ohe[i:i+50],
                                                   labels: np.argmax(Y_train[i:i+50], 1)})